In [1]:
import pandas as pd
import requests
import json

from datetime import datetime
import glob
from enum import Enum
base = pd.read_excel("./ExM Sample Tracking.xlsx", sheet_name="ExM Tissue Processing", header=[0,1,2], converters={})


In [2]:
with pd.option_context("display.multi_sparse", False):
    print(base.head)
    

<bound method NDFrame.head of              priority                  #  \
   Unnamed: 0_level_1 Unnamed: 1_level_1   
   Unnamed: 0_level_2 Unnamed: 1_level_2   
0                 NaN                1.0   
1                 NaN                2.0   
2                 NaN                3.0   
3                 NaN                4.0   
4                 NaN                5.0   
..                ...                ...   
85                NaN               82.0   
86                NaN               83.0   
87                NaN               84.0   
88                NaN               85.0   
89                NaN               86.0   

                                        Status      Processed By:  \
                            Unnamed: 2_level_1 Unnamed: 3_level_1   
                            Unnamed: 2_level_2 Unnamed: 3_level_2   
0                                          NaN                NDO   
1                                          NaN                NDO   
2       

In [6]:
subj_procedures = {}

download_files = True


if download_files:
    for row_idx, row in base.iterrows():    

        sample_id = str(row['Sample']['Unnamed: 4_level_1']['Unnamed: 4_level_2'])
        print(sample_id)
        
        if len(sample_id) != 6:
            continue

        # INFO: This originally loaded in the procedures files, but is not necessary at this moment since i have them saved. Prior block now loads these files into same structure.
        request = requests.get(f"http://aind-metadata-service/procedures/{sample_id}")
        #TODO: check for invalid/missing models

        if request.status_code == 404:
            print(f"{sample_id} model not found")

        item = request.json()
        print(item['message'])

        if item['message'] == 'Valid Model.':
            with open(f'./original_spec_files/{sample_id}.json', 'w') as outfile:
                json.dump(item['data'], outfile)
        else:
            with open(f'./original_spec_files/{sample_id}_invalid.json', 'w') as outfile:
                json.dump(item['data'], outfile)

576404
Validation Errors: 1 validation error for Procedures
subject_procedures.0.Surgery.procedures.0.Perfusion.protocol_id
  Field required [type=missing, input_value={'procedure_type': 'Perfu...ecimen_ids': {'576404'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
609105
Validation Errors: 5 validation errors for Procedures
subject_procedures.0.Surgery.procedures.0.Retro-orbital injection.injection_materials
  List should have at least 1 item after validation, not 0 [type=too_short, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.6/v/too_short
subject_procedures.0.Surgery.procedures.0.Retro-orbital injection.protocol_id
  Field required [type=missing, input_value={'injection_materials': [..., 'injection_eye': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
subject_procedures.0.Surgery.procedures.0.Retro-orbital injection.injection_volume

In [4]:
for row_idx, row in base.iterrows():
    sample_id = str(row['Sample'])
    genotype = row['Genotype']
    label = row['Label']
    inj_age = row['Age of Injection']
    post_fix = row['Post-Fix']
    
